# Load Dataset

In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset=pd.read_excel("C:/Users/10253/Desktop/ML-CW-2/Data/trainDataset.xls")
dataset.head(len(dataset)-1)

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.000000,41.0,0,0.0,0.0,1.0,3,3,...,0.517172,0.375126,3.325332,0.002314,3.880772e+06,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.000000,39.0,1,1.0,0.0,0.0,3,3,...,0.444391,0.444391,3.032144,0.005612,2.372010e+06,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.000000,31.0,0,0.0,0.0,1.0,2,1,...,0.534549,0.534549,2.485848,0.006752,1.540027e+06,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.000000,35.0,0,0.0,0.0,1.0,3,3,...,0.506185,0.506185,2.606255,0.003755,6.936741e+06,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.000000,61.0,1,0.0,0.0,0.0,2,1,...,0.462282,0.462282,2.809279,0.006521,1.265399e+06,39.621023,0.006585,0.034148,0.001083,0.005626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,TRG002954,0,49.250000,34.3,0,0.0,0.0,1.0,3,3,...,0.418382,0.418382,2.995603,0.004243,1.005061e+06,156.627179,0.002228,0.136015,0.022148,0.002098
395,TRG002955,1,49.250000,46.1,0,0.0,0.0,1.0,3,3,...,0.439568,0.439568,3.056046,0.001339,1.671271e+07,79.989003,0.003282,0.024716,0.000812,0.003078
396,TRG002958,0,48.500000,53.3,0,0.0,0.0,1.0,2,1,...,0.527779,0.527778,1.500000,0.003728,2.132007e+05,0.996746,0.252582,0.007380,0.000037,0.231059
397,TRG002961,0,47.500000,68.8,1,0.0,0.0,0.0,3,3,...,0.313693,0.313693,3.573557,0.001112,2.008034e+07,204.864200,0.001372,0.054063,0.003697,0.001368


In [34]:
# from sklearn.impute import  SimpleImputer
# imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# names=dataset.columns.drop('ID')
# a=imp.fit_transform(dataset.drop('ID',axis=1))
# imp=SimpleImputer(missing_values=999,strategy='mean')
# a=imp.fit_transform(a)
# df=pd.DataFrame(a,columns=
#                 names)
# df

# Data Preprocessing

### Removing duplicate

In [35]:
dataset.drop_duplicates(inplace=True) #because with 119 features, it is unlikely that exact same data happens in real world, it is useless for training and testing; hence remove it before dataset split

### Split Dataset

In [36]:
dataset['pCR (outcome)'].value_counts()

0      299
1       96
999      5
Name: pCR (outcome), dtype: int64

In [37]:
y=dataset['pCR (outcome)']
X=dataset.drop(['pCR (outcome)'],axis=1)


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

### drop columns that are not applicable

In [38]:
class columnDropperTransformer(): #https://stackoverflow.com/questions/68402691/adding-dropping-column-instance-into-a-pipeline
    def __init__(self,columns):
        self.columns=columns

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit_transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

dropper = columnDropperTransformer(['ID'])
X_train=dropper.fit_transform(X_train)

X_test=dropper.transform(X_test)

### missing data

In [39]:
dataset.isnull().sum()

ID                               0
pCR (outcome)                    0
RelapseFreeSurvival (outcome)    0
Age                              0
ER                               0
                                ..
original_ngtdm_Busyness          0
original_ngtdm_Coarseness        0
original_ngtdm_Complexity        0
original_ngtdm_Contrast          0
original_ngtdm_Strength          0
Length: 120, dtype: int64

In [40]:
from sklearn.impute import SimpleImputer

names=X_train.columns

imp=SimpleImputer(missing_values=np.nan,strategy='mean')
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

imp.missing_values=np.float64(999)
# X_train[:]=imp.fit_transform(X_train)
# X_test[:]=imp.transform(X_test)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

### Removing outliers

In [41]:
from sklearn.neighbors import LocalOutlierFactor

# because use LocalOutlierFactor may lose useful information becuase whenever there is an outlier in any features of the sample, the sample will be detected as outlier sample and all its information will be set to 999
# therefore, use other method

class outlierHandlingWithDev():
    def fit(self,X,y=None):
        self.mean=np.zeros(len(X[0]))
        self.std=np.zeros(len(X[0]))
        for i in range(len(X[0])):
            self.mean[i]=X[:,i].mean()
            self.std[i]=X[:,i].std()

        return self

    def transform(self,X,y=None):
        for i in range(len(X)):
            for j in range(len(X[i])):
                if np.abs(X[i][j]-self.mean[j])>3*self.std[j]:
                    X[i][j]=999
        return X

    def fit_transform(self,X,y=None):
        self.fit(X,y)
        return self.transform(X,y)
    # def fit(self,X,y=None):
    #     self.mean=X.mean(axis=0)
    #     self.std=X.std(axis=0)
    #
    #     return self
    #
    # def transform(self,X,y=None):
    #     for i in X.columns:
    #         for j in range(len(X)):
    #             if abs(X[i][j]-self.mean[i])>(3*self.std[i]):
    #                 X[i][j]=999
    #
    #     return X
    #
    # def fit_transform(self,X,y=None):
    #     self.fit(X,y)
    #     return self.transform(X,y)

detector=outlierHandlingWithDev()
detector.fit(X_train)
detector.transform(X_train)
detector.transform(X_test)

X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

X_train=pd.DataFrame(X_train,columns=names)
X_test=pd.DataFrame(X_test,columns=names)

### Binning

### resample

### MinMaxScaler

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train[:]=scaler.fit_transform(X_train)
X_test[:]=scaler.transform(X_test)
X_train

,RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,0.423077,0.781823,1.0,1.0,0.0,0.0,0.500000,0.0,1.000000,0.0,...,0.868388,0.924559,0.097225,0.040159,0.258104,0.004369,0.044803,0.003604,0.000012,0.178582
1,0.200000,0.282181,0.0,0.0,0.0,1.0,1.000000,1.0,0.000000,0.0,...,0.570113,0.606989,0.187005,0.309210,0.000669,0.005674,0.036042,0.177761,0.034518,0.150981
2,0.192308,0.778676,1.0,1.0,1.0,0.0,0.500000,0.0,1.000000,0.0,...,0.417328,0.444322,0.770089,0.358040,0.005766,0.120549,0.001589,0.411929,0.207050,0.007724
3,0.284615,0.605794,0.0,0.0,0.0,1.0,0.705036,1.0,0.125899,1.0,...,0.394987,0.420537,0.893701,0.437616,0.010137,0.177797,0.000935,0.318571,0.113167,0.004918
4,0.553846,0.219265,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,1.0,...,0.201109,0.214117,0.824875,0.159498,0.013977,0.119171,0.001656,0.346320,0.191612,0.008379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,0.438462,0.464348,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,0.0,...,0.431626,0.459545,0.491264,0.072624,0.038538,0.007283,0.026871,0.022532,0.000409,0.120550
276,0.561538,0.361702,0.0,0.0,1.0,0.0,0.500000,0.0,0.000000,0.0,...,0.615611,0.655431,0.268247,0.083550,0.026050,0.002509,0.078333,0.009931,0.000086,0.320601
277,0.384872,0.285753,1.0,1.0,0.0,0.0,1.000000,1.0,0.000000,1.0,...,0.506498,0.539260,0.679555,0.537708,0.005987,0.062738,0.002999,0.171437,0.032099,0.012846
278,0.269231,0.585795,1.0,1.0,1.0,0.0,0.500000,0.0,0.000000,0.0,...,0.309016,0.329003,0.428637,0.658642,0.000256,0.002968,0.067911,0.134257,0.016257,0.282100


### PCA

In [75]:
def is_related(pair1, pair2):
    if (pair1[0] in pair2 or pair1[1] in pair2) and pair1!=pair2 and not (pair1[0]==pair2[1] and pair1[1]==pair2[0]):
        return True
    return False

def add_relation(corr_fea):
    for i in range(len(corr_fea)):
        for j in range(i,len(corr_fea)):
            if i!=j:
                if is_related(corr_fea[i],corr_fea[j]):
                    fea1=tuple(set(corr_fea[i])-set(corr_fea[j]))
                    fea2=tuple(set(corr_fea[j])-set(corr_fea[i]))
                    fea=fea1+fea2
                    if fea not in corr_fea and rev(fea) not in corr_fea:
                        corr_fea.append((fea1+fea2))

def rev(fea):
    f1=fea[0]
    f2=fea[1]

    return (f2,f1)

corr=X_train.corr()
corr=corr.where(abs(corr)>0.9)
count=0
corr_fea=[]
for i in corr.columns:
    for j in corr.columns:
        if i!=j and abs(corr[i][j])>0.9 and ((i,j) not in corr_fea) and ((j,i) not in corr_fea):
            count+=1
            corr_fea.append((i,j))
            add_relation(corr_fea)

In [31]:
from sklearn.decomposition import PCA

new_num_fea=len(X_train.columns)-count

pca=PCA(n_components=new_num_fea)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)


array([[ 1.8129337 , -1.41678085, -0.57774469, ..., -0.01605587,
         0.01737679, -0.08038422],
       [-0.13334338,  0.27640146,  0.17228772, ...,  0.0123645 ,
        -0.02870864,  0.00961585],
       [ 2.17243919,  0.52860527, -0.19292474, ..., -0.03203643,
        -0.0055941 ,  0.01828341],
       ...,
       [-1.50563201, -0.28248341, -1.10582445, ...,  0.08881695,
        -0.02972668,  0.02484428],
       [-0.91025557,  0.07912714, -0.31998867, ...,  0.00239143,
         0.03134713,  0.02095628],
       [ 0.29819757,  0.09602321,  0.88729159, ..., -0.00979202,
         0.02277171, -0.00978471]])

### Feature Selection

In [76]:
from sklearn.feature_selection import SelectKBest
selector=SelectKBest(k=20)
X_train=selector.fit_transform(X_train,y_train)
X_test=selector.transform(X_test)
X_train

# from sklearn.feature_selection import S

array([[1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        3.60723453e-01, 9.72254357e-02, 1.18408663e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.87417999e-02, 1.87004819e-01, 3.45180721e-02],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        3.61882685e-01, 7.70089499e-01, 2.07050154e-01],
       ...,
       [1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        4.43686056e-01, 6.79554764e-01, 3.20987055e-02],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        3.49067984e-02, 4.28636707e-01, 1.62569988e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.01275840e-01, 4.66995864e-01, 9.00033502e-05]])

In [ ]:

# c=dataset.corr().abs()
# s=c.unstack()
# so=s.sort_values(kind="quicksort")





# delcol=[]
# c=dataset.corr().shape[0]

# names=dataset.corr().columns
#
# for i in range(c):
#     for j in range(i+1,c):
#         if dataset.corr().iloc[i,j]>0.8 :
#             delcol.append((names[i],names[j]))
# print(len(delcol))
#
# import numpy as np
# np.a
#
# for i in range(c):
#     count = 0
#     for j in range(c):
#         if dataset.corr().iloc[i,j]>0.8:
#             count += 1





# Feature Preprocession

# Methods